Often when I browse different publically available datasets on for example Kaggle I find most of them are already tidy. Sure, to use the data for machine learning you must do feature engineering, but most of the data cleaning is already done. I believe that this bad, since most of the data in the "real" world is messy. So, in this post, I will use R to clean messy data from the IMF data homepage (International Financial Statistics) with the ultimate goal of producing a tidy dataframe. I will use the same data as I used in the post [Python - Cleaning excel files from the International Monetary Fund](https://htp84.github.io/blog/posts/python-cleaning-excel-files-from-imf/). 

I have downloaded 8 excel files and they contain the following data:

* Financial Market Prices, Equities, End of Period, Index
* Prices, Consumer Price Index, All items, Index
* Prices, Producer Price Index, All Commodities, Index
* Economic Activity, Industrial Production, Index
* Labor Force, Persons, Number of
* Unemployment, Persons, Number of
* Employment, Persons, Number of
* Labor Markets, Unemployment Rate, Percent

The post consists of the following four parts:

1. Setup and the first glance at a file
2. Parse and clean all files
3. Merge with external data
4. Save data

## 1. Setup and a first look at one file
First I load the libraries I need. Below I list som examples of what I use the libraries for in this post. Follow the links to see more use cases. 
* [dplyr](https://dplyr.tidyverse.org/) is used manipulate data, e.g. add new variables and select variables.
* [tidyr](https://tidyr.tidyverse.org/) is used to create tidy data. 
* [stringr](https://stringr.tidyverse.org/) consists of functions that make working with strings as easy as possible.
* [purrr](https://purrr.tidyverse.org/)
* [tibble](https://tibble.tidyverse.org/)
* [readxl](https://readxl.tidyverse.org/) makes it easy to get data out of Excel and into R.
* [readr](https://readr.tidyverse.org/)
* [janitor](https://github.com/sfirke/janitor) is used to format column names.

I also create a custom cat (print) function to add newlines. This is used to add explanations to output.

In [2]:
options(warn=-1)
library(dplyr)
library(tidyr)
library(stringr)
library(purrr)
library(tibble)
library(readxl)
library(readr)
library(janitor);

In [3]:
catn <- function(x){
    cat("\n\n", x, "\n\n", sep = "")
}

shape <- function(df){
    cat("\n\n", dim(df)[1],",", dim(df)[2], sep="")
}

To see which excel files I have in the current directory I use the [list.files](https://stat.ethz.ch/R-manual/R-devel/library/base/html/list.files.html) function. The files are named poorly, as seen below. However, I know there is information in the files that will help to identify them. Therefore, I parse the first file in files. I view its first 10 rows using the head function.

In [4]:
catn("Files:")
files <- list.files(pattern = "\\.xlsx$")
files

catn("Data from the first file:")
head(read_excel(files[1]))



Files:



[1] "Prices_Production_and_Labor (1).xlsx"
[2] "Prices_Production_and_Labor (2).xlsx"
[3] "Prices_Production_and_Labor (3).xlsx"
[4] "Prices_Production_and_Labor (4).xlsx"
[5] "Prices_Production_and_Labor (5).xlsx"
[6] "Prices_Production_and_Labor (6).xlsx"
[7] "Prices_Production_and_Labor (7).xlsx"
[8] "Prices_Production_and_Labor.xlsx"



Data from the first file:



New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems


"Prices, Production and Labor selected indicators",...2,...3,...4,...5,...6,...7,...8,...9,...10,⋯,...96,...97,...98,...99,...100,...101,...102,...103,...104,...105
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Prices, Consumer Price Index, All items, Index","getSelectionEl(0,1,""Indicator"")","getSelectionEl(0,1,""Indicator"")",NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Source: International Financial Statistics (IFS),NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Country,Scale,Base Year,2013,2013Q1,2013M01,2013M02,2013M03,2013Q2,2013M04,⋯,2018M05,2018M06,2018Q3,2018M07,2018M08,2018M09,2018Q4,2018M10,2018M11,2018M12
"Afghanistan, Islamic Republic of",Units,2010=100,127.795223073732,125.223096076787,125.191195214315,125.515928270858,124.962164745187,126.716990862207,126.997097858061,⋯,145.34294838289301,145.19114224173501,145.51246349769201,145.38508863027101,145.297193117007,145.855108745797,...,147.25501270754501,148.21762074521999,...


The interesting data starts at index 5 in the Unnamed:1 column. The first column does not include data so it is unnecessary. The row at index 5 is the header. Also, the KPI ID is at index 1 column Unnamed:1. This information is useful.

I will import the data again and then clean the dataframe. In the cell below I explain the purpose of every line.

In [5]:
data <- read_excel(files[1]) # parse the excel file
kpi <- str_c(data[1, 1])  # get the kpi name
df <- data[-(1:5),] # get the data
cols <- data[5,] # get columnnames
colnames(df) = cols # set columnnames
df <- df %>% add_column(kpi, .after=1) # add kpi at position 1
catn("Cleaned data v1 (one file) and shape:")
head(df)
shape(df)

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems




Cleaned data v1 (one file) and shape:



Country,kpi,Scale,Base Year,2013,2013Q1,2013M01,2013M02,2013M03,2013Q2,⋯,2018M05,2018M06,2018Q3,2018M07,2018M08,2018M09,2018Q4,2018M10,2018M11,2018M12
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Afghanistan, Islamic Republic of","Prices, Consumer Price Index, All items, Index",Units,2010=100,127.795223073732,125.223096076787,125.191195214315,125.515928270858,124.962164745187,126.716990862207,⋯,145.34294838289301,145.19114224173501,145.51246349769201,145.38508863027101,145.297193117007,145.855108745797,...,147.25501270754501,148.21762074521999,...
Albania,"Prices, Consumer Price Index, All items, Index",Units,2010=100,107.58166295471401,108.46325167037899,107.57238307349699,108.79732739420901,109.02004454343,107.943578322197,⋯,116.843122494432,116.777638084633,116.844479955457,116.470825167038,116.896940979955,117.165673719376,117.18271752041601,116.963307349666,116.67275055679301,117.91209465478801
Algeria,"Prices, Consumer Price Index, All items, Index",Units,2010=100,117.52183806797299,117.773862830018,117.69067019012,117.514497540923,118.11642075901101,117.345665418777,⋯,149.533876532335,151.15613301035,148.865888570799,148.25662482566301,148.93195331424801,149.409087572488,...,151.19283564559899,150.51750715701399,...
Angola,"Prices, Consumer Price Index, All items, Index",Units,2010=100,136.13117928470399,132.31043006940899,131.29906311983601,132.378692172749,133.253534915644,135.11455434111701,⋯,329.14408793675602,332.96174100655298,342.88282726555701,337.05145577843803,341.11130241019299,350.48572360804201,...,355.286581821288,360.20852718653498,...
Anguilla,"Prices, Consumer Price Index, All items, Index",Units,2010=100,106.355019921305,106.065480462273,...,...,...,106.62971120295001,⋯,...,...,...,...,...,...,...,...,...,...
Antigua and Barbuda,"Prices, Consumer Price Index, All items, Index",Units,2010=100,108.083497160332,108.14220183486201,108.232306684142,108.346985583224,107.847313237222,108.073940585408,⋯,113.83519003931799,113.990825688073,114.05362603757099,113.83519003931799,114.162844036697,114.162844036697,...,...,...,...




203,106

As can be seen in the last row above, missing data is coded with three dots (...). This must be replaced with NaN. Also, in my opinion, the kpi column and all column names will look better if comma and spaces were replaced with underscores and if they were set to lowercase. Since I have added the kpi column I need to remove it again before renaming the columns. Otherwise, there will be a Value Error since there is a length mismatch between the cols list and the df columns. I reset the kpi column using the kpi variable set in the previous code cell. To clean the column names I use the [make_clean_names and clean_names](https://cran.r-project.org/web/packages/janitor/vignettes/janitor.html) functions from janitor.

In [6]:
df <- df %>% select(-c("kpi")) %>% clean_names()
kpi <- kpi %>% make_clean_names()
df <- df %>% add_column(kpi, .after=1) %>%
  na_if("...")
catn("Cleaned data v2 (one file):")
head(df)



Cleaned data v2 (one file):



country,kpi,scale,base_year,x2013,x2013q1,x2013m01,x2013m02,x2013m03,x2013q2,⋯,x2018m05,x2018m06,x2018q3,x2018m07,x2018m08,x2018m09,x2018q4,x2018m10,x2018m11,x2018m12
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Afghanistan, Islamic Republic of",prices_consumer_price_index_all_items_index,Units,2010=100,127.795223073732,125.223096076787,125.191195214315,125.515928270858,124.962164745187,126.716990862207,⋯,145.34294838289301,145.19114224173501,145.51246349769201,145.38508863027101,145.297193117007,145.855108745797,NA,147.25501270754501,148.21762074521999,NA
Albania,prices_consumer_price_index_all_items_index,Units,2010=100,107.58166295471401,108.46325167037899,107.57238307349699,108.79732739420901,109.02004454343,107.943578322197,⋯,116.843122494432,116.777638084633,116.844479955457,116.470825167038,116.896940979955,117.165673719376,117.18271752041601,116.963307349666,116.67275055679301,117.91209465478801
Algeria,prices_consumer_price_index_all_items_index,Units,2010=100,117.52183806797299,117.773862830018,117.69067019012,117.514497540923,118.11642075901101,117.345665418777,⋯,149.533876532335,151.15613301035,148.865888570799,148.25662482566301,148.93195331424801,149.409087572488,NA,151.19283564559899,150.51750715701399,NA
Angola,prices_consumer_price_index_all_items_index,Units,2010=100,136.13117928470399,132.31043006940899,131.29906311983601,132.378692172749,133.253534915644,135.11455434111701,⋯,329.14408793675602,332.96174100655298,342.88282726555701,337.05145577843803,341.11130241019299,350.48572360804201,NA,355.286581821288,360.20852718653498,NA
Anguilla,prices_consumer_price_index_all_items_index,Units,2010=100,106.355019921305,106.065480462273,NA,NA,NA,106.62971120295001,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Antigua and Barbuda,prices_consumer_price_index_all_items_index,Units,2010=100,108.083497160332,108.14220183486201,108.232306684142,108.346985583224,107.847313237222,108.073940585408,⋯,113.83519003931799,113.990825688073,114.05362603757099,113.83519003931799,114.162844036697,114.162844036697,NA,NA,NA,NA


## 2. Parse and clean all files

Now the dataframe looks pretty good. But I have only imported one excel file. Let's summarise the steps above in one cell and parse all excel files. This can be done with a for loop iterating over the files list.

In [7]:
financial_data <- tibble()
for (i in files){
  data <- read_excel(i)
  kpi <- str_c(data[1, 1]) %>% 
    make_clean_names()
  cols <- data[5,] %>% 
    make_clean_names()
  colnames(data) = cols
  df <- data[-(1:5),] %>% 
    add_column(kpi, .after=1) %>% 
    na_if("...")
  financial_data <- bind_rows(financial_data, df)
  #head(df)
}
colnames(financial_data) <- colnames(df)

catn("Cleaned data v1 (all files) and shape:")
sample_n(financial_data, 10)
shape(financial_data)

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* … and 99 more problems




Cleaned data v1 (all files) and shape:



country,kpi,scale,base_year,x2013,x2013q1,x2013m01,x2013m02,x2013m03,x2013q2,⋯,x2018m05,x2018m06,x2018q3,x2018m07,x2018m08,x2018m09,x2018q4,x2018m10,x2018m11,x2018m12
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Eswatini, Kingdom of",prices_consumer_price_index_all_items_index,Units,2010=100,122.09040253117,119.99946309364501,118.83976446277801,120.505386215811,120.653238602345,121.737608003397,⋯,163.94451474232901,163.81407877331799,164.13674707866201,163.991001789319,164.08477454748501,164.334464899181,164.96166674413999,164.75101432130799,165.01966920170699,165.11431670940399
Germany,labor_force_persons_number_of,Thousands,NA,44453.416666666701,44265,44259,44318,44218,44352.666666666701,⋯,46139,46228,NA,46184,46274,NA,NA,NA,NA,NA
United Arab Emirates,prices_consumer_price_index_all_items_index,Units,2010=100,102.663561221631,102.03838610255001,102.097926590081,101.95082656206201,102.066405155506,102.31507425049,⋯,116.71336508826001,117.15349274632401,116.971577834285,117.094607503872,117.264857792177,116.55526820680601,115.65137520430901,115.99923245229201,115.65452337187899,115.300369788757
Curaçao,unemployment_persons_number_of,Thousands,NA,10,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Russian Federation,employment_persons_number_of,Thousands,NA,71391.666666666701,70900,70700,71000,71000,71400,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
United Kingdom,employment_persons_number_of,Thousands,NA,30050.568500000001,29777.089333333297,29771.93,29750.279999999999,29809.058000000001,29872.7103333333,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Tanzania,employment_persons_number_of,Thousands,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Mali,employment_persons_number_of,Thousands,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
New Zealand,prices_consumer_price_index_all_items_index,Units,2010=100,106.323136529614,105.670575668258,NA,NA,NA,105.85059131603499,⋯,NA,NA,112.999462590446,NA,NA,NA,113.109813628132,NA,NA,NA




860,106

All excel files are now parsed and appended to the financial_data dataframe. But the dataframe is still not easy to analyze. Every period has its own column. Period is a variable so it should only be one column. Hence, the dataframe must be unpivoted/melted/gathered. This can be done with the [gather](https://tidyr.tidyverse.org/reference/gather.html) function from tidyr. I also use the unique function to see which periods are included.

In [8]:
cols <- c("country", "kpi", "scale", "base_year")
financial_data <- financial_data %>% gather(key = "period", value = "value", -cols)

catn("Unique periods:")
print(unique(financial_data$period))

catn("Cleaned data v2 (all files) and shape:")
sample_n(financial_data, 10)
shape(financial_data)



Unique periods:

  [1] "x2013"    "x2013q1"  "x2013m01" "x2013m02" "x2013m03" "x2013q2" 
  [7] "x2013m04" "x2013m05" "x2013m06" "x2013q3"  "x2013m07" "x2013m08"
 [13] "x2013m09" "x2013q4"  "x2013m10" "x2013m11" "x2013m12" "x2014"   
 [19] "x2014q1"  "x2014m01" "x2014m02" "x2014m03" "x2014q2"  "x2014m04"
 [25] "x2014m05" "x2014m06" "x2014q3"  "x2014m07" "x2014m08" "x2014m09"
 [31] "x2014q4"  "x2014m10" "x2014m11" "x2014m12" "x2015"    "x2015q1" 
 [37] "x2015m01" "x2015m02" "x2015m03" "x2015q2"  "x2015m04" "x2015m05"
 [43] "x2015m06" "x2015q3"  "x2015m07" "x2015m08" "x2015m09" "x2015q4" 
 [49] "x2015m10" "x2015m11" "x2015m12" "x2016"    "x2016q1"  "x2016m01"
 [55] "x2016m02" "x2016m03" "x2016q2"  "x2016m04" "x2016m05" "x2016m06"
 [61] "x2016q3"  "x2016m07" "x2016m08" "x2016m09" "x2016q4"  "x2016m10"
 [67] "x2016m11" "x2016m12" "x2017"    "x2017q1"  "x2017m01" "x2017m02"
 [73] "x2017m03" "x2017q2"  "x2017m04" "x2017m05" "x2017m06" "x2017q3" 
 [79] "x2017m07" "x2017m08" "x2017m09" "x2017

country,kpi,scale,base_year,period,value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Slovak Republic,prices_consumer_price_index_all_items_index,Units,2010=100,x2016m03,108.032596041909
"Venezuela, Republica Bolivariana de",prices_consumer_price_index_all_items_index,Units,2010=100,x2016m02,1480.6272575103501
Tunisia,labor_markets_unemployment_rate_percent,Units,NA,x2015m02,NA
Italy,prices_producer_price_index_all_commodities_index,Units,2010=100,x2015m09,103.77194507213601
Mongolia,labor_force_persons_number_of,Thousands,NA,x2013m05,NA
Moldova,labor_markets_unemployment_rate_percent,Units,NA,x2014m04,NA
Palau,prices_consumer_price_index_all_items_index,Units,2010=100,x2016q4,117.204819277108
South Africa,prices_consumer_price_index_all_items_index,Units,2010=100,x2014m09,126.091586794462
Slovak Republic,prices_consumer_price_index_all_items_index,Units,2010=100,x2018m09,112.755695992017




87720,6

The periods include quarters (e.g. 2013q1), year (e.g. 2015) and months (e.g. 2018m06). There is one problem, ther is a 'x' in front of all rows in period column. This problem is caused by gather but is easily fixed with [str_replace](https://stringr.tidyverse.org/reference/str_replace.html) from stringr.  I also set the value column to numeric and count the number of null values in the value column.

In [9]:
financial_data <- financial_data %>% mutate(period = str_replace(period, "x", ""))
financial_data$value <- as.numeric(financial_data$value)

catn("Control periods:")
print(unique(financial_data$period))

catn("Number of nulls in the value column:")
sum(is.na(financial_data$value))



Control periods:

  [1] "2013"    "2013q1"  "2013m01" "2013m02" "2013m03" "2013q2"  "2013m04"
  [8] "2013m05" "2013m06" "2013q3"  "2013m07" "2013m08" "2013m09" "2013q4" 
 [15] "2013m10" "2013m11" "2013m12" "2014"    "2014q1"  "2014m01" "2014m02"
 [22] "2014m03" "2014q2"  "2014m04" "2014m05" "2014m06" "2014q3"  "2014m07"
 [29] "2014m08" "2014m09" "2014q4"  "2014m10" "2014m11" "2014m12" "2015"   
 [36] "2015q1"  "2015m01" "2015m02" "2015m03" "2015q2"  "2015m04" "2015m05"
 [43] "2015m06" "2015q3"  "2015m07" "2015m08" "2015m09" "2015q4"  "2015m10"
 [50] "2015m11" "2015m12" "2016"    "2016q1"  "2016m01" "2016m02" "2016m03"
 [57] "2016q2"  "2016m04" "2016m05" "2016m06" "2016q3"  "2016m07" "2016m08"
 [64] "2016m09" "2016q4"  "2016m10" "2016m11" "2016m12" "2017"    "2017q1" 
 [71] "2017m01" "2017m02" "2017m03" "2017q2"  "2017m04" "2017m05" "2017m06"
 [78] "2017q3"  "2017m07" "2017m08" "2017m09" "2017q4"  "2017m10" "2017m11"
 [85] "2017m12" "2018"    "2018q1"  "2018m01" "2018m02" "2018m03" "2

[1] 32141

There are 32141 rows with null values. One should always be careful when handling null values. But in this case, they are just in the way, so I will drop them and then control if there are any left.

The period column is not optimal, it would be nice if one easily could distinguish the time period. Also, it would be nice if year and period (month, quarter) were separate columns. This can be accomplished with the custom functions.

I do all the above operations based on the period column. Then I replace the old period column with the new column that specifies the current month/quarter. NaN is set if the data is yearly.

In [10]:
columns <- c("country", "kpi", "value")
year <- substr(financial_data$period, start = 1, stop = 4)

extract_time_period_value <- function(x) {
  if (grepl("q", x)) {
    time_period <- str_sub(x, start = -1)
  } else if (grepl("m", x)) {
    time_period <- str_sub(x, start = -2)
  } else {
    time_period <- NA
  }
}
extract_time_period <- function(x) {
  if (grepl("q", x)) {
    time_period <- "quarter"
  } else if (grepl("m", x)) {
    time_period <- "month"
  } else {
    time_period <- "year"
  }
}

period <- map_chr(financial_data$period, extract_time_period_value)
time_period <- map_chr(financial_data$period, extract_time_period)

financial_data <- financial_data %>%
  select(columns) %>% 
  add_column(year, .after=2) %>% 
  add_column(period, .after=3) %>% 
  add_column(time_period, .after=4) %>% 
  drop_na(value)

financial_data$value <- as.numeric(financial_data$value)

catn("Cleaned data v3 (all files):")
sample_n(financial_data,10)



Cleaned data v3 (all files):



country,kpi,year,period,time_period,value
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Latvia,unemployment_persons_number_of,2018,08,month,71.00000
Iceland,labor_force_persons_number_of,2015,NA,year,191.36667
Bosnia and Herzegovina,prices_producer_price_index_all_commodities_index,2017,01,month,103.57303
Greece,employment_persons_number_of,2015,09,month,3634.55836
Montenegro,labor_markets_unemployment_rate_percent,2015,1,quarter,18.16065
Singapore,financial_market_prices_equities_end_of_period_index,2015,01,month,114.76333
Israel,economic_activity_industrial_production_index,2016,1,quarter,112.43409
Vietnam,prices_consumer_price_index_all_items_index,2017,07,month,154.46347
Tunisia,prices_producer_price_index_all_commodities_index,2018,06,month,145.95121


Now there are two steps left until the dataframe is tidy. First, the kpi column needs to be spread/pivoted. This because the kpi column contains multiple kpis and every kpi is a variable. For this, I will use the [spread](https://tidyr.tidyverse.org/reference/spread.html) function from tidyr.

Also, the dataframe contains three different time periods (year, month, quarter). The time periods need to be split into three separate dataframes.

## 3. Merge with external data
However, it would help the analysis if the dataframe contained more information about the country, for example, region. This link contains a csv with countries and their region. Let's look at the data.

In [11]:
region <- read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv") %>% 
  clean_names()
catn("Regions:")
sample_n(region, 10)

catn("Regions and sub-regions:")
region %>% select("region", "sub_region") %>% 
  distinct() %>%
  arrange(region, sub_region)

Parsed with column specification:
cols(
  name = col_character(),
  `alpha-2` = col_character(),
  `alpha-3` = col_character(),
  `country-code` = col_character(),
  `iso_3166-2` = col_character(),
  region = col_character(),
  `sub-region` = col_character(),
  `intermediate-region` = col_character(),
  `region-code` = col_character(),
  `sub-region-code` = col_character(),
  `intermediate-region-code` = col_character()
)




Regions:



name,alpha_2,alpha_3,country_code,iso_3166_2,region,sub_region,intermediate_region,region_code,sub_region_code,intermediate_region_code
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cuba,CU,CUB,192,ISO 3166-2:CU,Americas,Latin America and the Caribbean,Caribbean,019,419,029
Sudan,SD,SDN,729,ISO 3166-2:SD,Africa,Northern Africa,NA,002,015,NA
United Kingdom of Great Britain and Northern Ireland,GB,GBR,826,ISO 3166-2:GB,Europe,Northern Europe,NA,150,154,NA
British Indian Ocean Territory,IO,IOT,086,ISO 3166-2:IO,Africa,Sub-Saharan Africa,Eastern Africa,002,202,014
Lesotho,LS,LSO,426,ISO 3166-2:LS,Africa,Sub-Saharan Africa,Southern Africa,002,202,018
Sint Maarten (Dutch part),SX,SXM,534,ISO 3166-2:SX,Americas,Latin America and the Caribbean,Caribbean,019,419,029
Paraguay,PY,PRY,600,ISO 3166-2:PY,Americas,Latin America and the Caribbean,South America,019,419,005
Sao Tome and Principe,ST,STP,678,ISO 3166-2:ST,Africa,Sub-Saharan Africa,Middle Africa,002,202,017
"Bonaire, Sint Eustatius and Saba",BQ,BES,535,ISO 3166-2:BQ,Americas,Latin America and the Caribbean,Caribbean,019,419,029




Regions and sub-regions:



region,sub_region
<chr>,<chr>
Africa,Northern Africa
Africa,Sub-Saharan Africa
Americas,Latin America and the Caribbean
Americas,Northern America
Asia,Central Asia
Asia,Eastern Asia
Asia,South-eastern Asia
Asia,Southern Asia
Asia,Western Asia


There are many columns, but I find the name, region and sub-region columns most interesting. The name column can be used as a key when joining with the financial_data dataframe. Optimal would have been if the original data had contained one of the other columns since it would have increased the likelihood for a perfect match. But I have to take what I have got and deal with it.

So, I filter the columns in the region dataframe and rename the name column to country. Then I use the [left_join](https://dplyr.tidyverse.org/reference/join.html) method from dplyr to join the financial_data and region dataframes using country as key. I create a new dataframe called financial_data2. This is a temporary dataframe, I will use it to see which countries in financial_data that do not have a match in the region dataframe.

In [12]:
region <- region %>% 
  select("name", "region", "sub_region") %>% 
  rename(country = name)
catn("Cleaned region data:")
sample_n(region, 10)

financial_data2 <- left_join(x=financial_data, y=region, by=c("country"))

catn("Countries with no match in region dataframe:")
financial_data2 %>% select(country, region) %>% 
  filter_all(any_vars(is.na(.))) %>% 
  distinct() %>% 
  rownames_to_column("index")



Cleaned region data:



country,region,sub_region
<chr>,<chr>,<chr>
Venezuela (Bolivarian Republic of),Americas,Latin America and the Caribbean
"Saint Helena, Ascension and Tristan da Cunha",Africa,Sub-Saharan Africa
Congo,Africa,Sub-Saharan Africa
Ethiopia,Africa,Sub-Saharan Africa
Barbados,Americas,Latin America and the Caribbean
South Georgia and the South Sandwich Islands,Americas,Latin America and the Caribbean
Equatorial Guinea,Africa,Sub-Saharan Africa
Maldives,Asia,Southern Asia
"Korea, Republic of",Asia,Eastern Asia




Countries with no match in region dataframe:



index,country,region
<chr>,<chr>,<chr>
1,"Afghanistan, Islamic Republic of",NA
2,"Armenia, Republic of",NA
3,"Azerbaijan, Republic of",NA
4,"Bahamas, The",NA
5,"Bahrain, Kingdom of",NA
6,Bolivia,NA
7,"China, P.R.: Hong Kong",NA
8,"China, P.R.: Macao",NA
9,"China, P.R.: Mainland",NA


There are 48 "countries" that do not have a match. But, as can be seen above, the "countries" from index 37 to 47 are not countries. They are regions. These can be removed. This is done in the missing dataframe below.

Also, many of the countries in the financial_data dataframe that have no match in the region dataframe have longer names, often split by a comma, e.g. Bahamas, The. Therefore I create a new column in the missing dataframe called country2 where I split the country name on comma (,).

In [13]:
missing <- financial_data2 %>%
  select(country, region) %>% 
  filter_all(any_vars(is.na(.))) %>% 
  distinct() %>% 
  select(country)
  
missing <- missing %>% bind_cols(missing %>% separate(country, c("country2"), ",")) %>% 
  rownames_to_column("index") 
missing$index <- as.numeric(missing$index)
  
missing <- missing %>% filter(index < 37 | index==48)
catn("Countries with no match v2:")
missing



Countries with no match v2:



index,country,country2
<dbl>,<chr>,<chr>
1,"Afghanistan, Islamic Republic of",Afghanistan
2,"Armenia, Republic of",Armenia
3,"Azerbaijan, Republic of",Azerbaijan
4,"Bahamas, The",Bahamas
5,"Bahrain, Kingdom of",Bahrain
6,Bolivia,Bolivia
7,"China, P.R.: Hong Kong",China
8,"China, P.R.: Macao",China
9,"China, P.R.: Mainland",China


Now I probably will get a match on for example Afghanistan, but many of the countries in the missing dataframe did not have a comma, hence I will not find a match for them. To solve this problem I will use the [agrep](https://www.rdocumentation.org/packages/base/versions/3.6.0/topics/agrep) function. This functions searches for approximate matches and returns a list of the best matches.

I map the result of agrep to a new column, country3, in the missing dataframe.

In [14]:
matches <- c()
for (i in missing$country2) {
  row <- agrep(i, region$country)[1]
  matches <- c(matches, region$country[row])
}
catn("Matched countries:")
matches

missing <- missing %>% mutate(country3 = matches)
catn("Countries with no match v3:")
missing



Matched countries:



[1] "Afghanistan"                                         
 [2] "Armenia"                                             
 [3] "Azerbaijan"                                          
 [4] "Bahamas"                                             
 [5] "Bahrain"                                             
 [6] "Bolivia (Plurinational State of)"                    
 [7] "China"                                               
 [8] "China"                                               
 [9] "China"                                               
[10] "Congo"                                               
[11] "Congo"                                               
[12] NA                                                    
[13] "Eswatini"                                            
[14] NA                                                    
[15] "Gambia"                                              
[16] "France"                                              
[17] NA                                                    
[18] NA                                                    
[19] "North Macedonia"                                     
[20] "Micronesia (Federated States of)"                    
[21] "Moldova, Republic of"                                
[22] "Sao Tome and Principe"                               
[23] "Serbia"                                              
[24] "Sint Maarten (Dutch part)"                           
[25] NA                                                    
[26] "Saint Kitts and Nevis"                               
[27] NA                                                    
[28] "Saint Vincent and the Grenadines"                    
[29] "Tanzania, United Republic of"                        
[30] "Timor-Leste"                                         
[31] "United Kingdom of Great Britain and Northern Ireland"
[32] "United States of America"                            
[33] "Venezuela (Bolivarian Republic of)"                  
[34] NA                                                    
[35] NA                                                    
[36] "Yemen"                                               
[37] NA



Countries with no match v3:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
1,"Afghanistan, Islamic Republic of",Afghanistan,Afghanistan
2,"Armenia, Republic of",Armenia,Armenia
3,"Azerbaijan, Republic of",Azerbaijan,Azerbaijan
4,"Bahamas, The",Bahamas,Bahamas
5,"Bahrain, Kingdom of",Bahrain,Bahrain
6,Bolivia,Bolivia,Bolivia (Plurinational State of)
7,"China, P.R.: Hong Kong",China,China
8,"China, P.R.: Macao",China,China
9,"China, P.R.: Mainland",China,China


Now it looks pretty good. But it is not perfect. There are some NA's and one error (Iran is not the same as France). I set the errors to NA. Then I create a new dataframe called corrected which contains all corrected countries. I then overwrite the missing dataframe with the countries that do have NaN in the country3 column.

In [15]:
missing[16, 4] <- NA #Iran
corrected <- drop_na(missing)
missing <- missing %>%  filter(is.na(country3))

catn("Corrected:")
corrected
catn("Countries with no match v4:")
missing



Corrected:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
1,"Afghanistan, Islamic Republic of",Afghanistan,Afghanistan
2,"Armenia, Republic of",Armenia,Armenia
3,"Azerbaijan, Republic of",Azerbaijan,Azerbaijan
4,"Bahamas, The",Bahamas,Bahamas
5,"Bahrain, Kingdom of",Bahrain,Bahrain
6,Bolivia,Bolivia,Bolivia (Plurinational State of)
7,"China, P.R.: Hong Kong",China,China
8,"China, P.R.: Macao",China,China
9,"China, P.R.: Mainland",China,China




Countries with no match v4:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
12,Czech Republic,Czech Republic,NA
14,French Territories: New Caledonia,French Territories: New Caledonia,NA
16,"Iran, Islamic Republic of",Iran,NA
17,"Kosovo, Republic of",Kosovo,NA
18,Kyrgyz Republic,Kyrgyz Republic,NA
25,Slovak Republic,Slovak Republic,NA
27,St. Lucia,St. Lucia,NA
34,Vietnam,Vietnam,NA
35,West Bank and Gaza,West Bank and Gaza,NA


There are only 10 countries left. Below I try to find a match using the [str_sub](https://stringr.tidyverse.org/reference/str_sub.html) function from stringr..  This functions works similarly to the LIKE keyword in SQL. I will only use the first 4 letters in the country2 column. The matches are stored in a character vector named matches. Then I replace the country3 column in the missing dataframe with this vector.

In [16]:
matches <- c()
for (i in missing$country2) {
  i_ <- str_sub(i,1,4)
  row <- agrep(i_, region$country,)[1]
  matches <- c(matches, region$country[row])
}

missing <- missing %>% mutate(country3=matches) %>% 
  select(-index) %>% 
  rownames_to_column("index")
missing$index <- as.numeric(missing$index)

catn("Countries with no match v5:")
missing



Countries with no match v5:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
1,Czech Republic,Czech Republic,Czechia
2,French Territories: New Caledonia,French Territories: New Caledonia,France
3,"Iran, Islamic Republic of",Iran,France
4,"Kosovo, Republic of",Kosovo,NA
5,Kyrgyz Republic,Kyrgyz Republic,Kyrgyzstan
6,Slovak Republic,Slovak Republic,Slovakia
7,St. Lucia,St. Lucia,NA
8,Vietnam,Vietnam,Viet Nam
9,West Bank and Gaza,West Bank and Gaza,"Palestine, State of"


In the output above it is visible that most countries find a correct match in the country3 column. However, French Territories: New Caledonia and Iran are not France and Netherlands Antilles is not Netherlands. Also, Kosovo and St. Lucia are NA. 

I replace the errors with NA and move the correct rows to the corrected dataframe. Then I remove all rows in the missing dataframe except for the NA's.

In [17]:
missing[2, 4] <- NA
missing[3, 4] <- NA
missing[10, 4] <- NA
corrected <- corrected %>% bind_rows(drop_na(missing)) %>% 
  select(-index) %>% 
  rownames_to_column("index")
corrected$index <- as.numeric(corrected$index)
  
missing <- missing %>% filter(is.na(country3))

catn("Corrected v2:")
corrected
catn("Countries with no match v6:")
missing



Corrected v2:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
1,"Afghanistan, Islamic Republic of",Afghanistan,Afghanistan
2,"Armenia, Republic of",Armenia,Armenia
3,"Azerbaijan, Republic of",Azerbaijan,Azerbaijan
4,"Bahamas, The",Bahamas,Bahamas
5,"Bahrain, Kingdom of",Bahrain,Bahrain
6,Bolivia,Bolivia,Bolivia (Plurinational State of)
7,"China, P.R.: Hong Kong",China,China
8,"China, P.R.: Macao",China,China
9,"China, P.R.: Mainland",China,China




Countries with no match v6:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
2,French Territories: New Caledonia,French Territories: New Caledonia,NA
3,"Iran, Islamic Republic of",Iran,NA
4,"Kosovo, Republic of",Kosovo,NA
7,St. Lucia,St. Lucia,NA
10,Netherlands Antilles,Netherlands Antilles,NA


Only 5 countries left! Now I will use agrep again, but instead of looping over all rows I will do one at a time. 

In [18]:
catn("New Caledonia")
region$country[agrep("Caled", region$country)]
catn("Iran")
region$country[agrep("Iran", region$country)]
catn("Kosovo")
region$country[agrep("Kos", region$country)]
catn("St.Lucia")
region$country[agrep("Luci", region$country)]
catn("Antilles 1")
region$country[agrep("Antill", region$country)]
catn("Antilles 2")
region$country[agrep("Cura", region$country)]



New Caledonia



[1] "New Caledonia"



Iran



[1] "France"                     "Iran (Islamic Republic of)"
[3] "Iraq"



Kosovo



...

[1] "Saint Lucia"



Antilles 1



character(0)



Antilles 2



[1] "Cuba"     "Curaçao"  "Honduras"

There are matches on all countries except Kosovo and Antilles 1. In antilles 2 I use Curacao since it is the  main island. For Kosovo I use Serbia as proxy. This is good alternative since I only use the country3 as a key to join the financial_data dataframe with the region dataframe.

In [19]:
missing[1, 4] <- "New Caledonia"
missing[2, 4] <- "Iran (Islamic Republic of)"
missing[3, 4] <- "Serbia"
missing[4, 4] <- "Saint Lucia"
missing[5, 4] <- "Curaçao"

corrected <- corrected %>% bind_rows(drop_na(missing)) %>% 
  select(-index) %>% 
  rownames_to_column("index")
corrected$index <- as.numeric(corrected$index)

missing <- missing %>%  filter(is.na(country3))

catn("Corrected v3:")
corrected
catn("Countries with no match v7:")
missing



Corrected v3:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>
1,"Afghanistan, Islamic Republic of",Afghanistan,Afghanistan
2,"Armenia, Republic of",Armenia,Armenia
3,"Azerbaijan, Republic of",Azerbaijan,Azerbaijan
4,"Bahamas, The",Bahamas,Bahamas
5,"Bahrain, Kingdom of",Bahrain,Bahrain
6,Bolivia,Bolivia,Bolivia (Plurinational State of)
7,"China, P.R.: Hong Kong",China,China
8,"China, P.R.: Macao",China,China
9,"China, P.R.: Mainland",China,China




Countries with no match v7:



index,country,country2,country3
<dbl>,<chr>,<chr>,<chr>


To get correct regions* in the region dataframe, I join the corrected dataframe with the region dataframe. I join country3 in corrected on country in region

In [20]:
corrected <- left_join(x=corrected, y=region, by = c("country3" = "country")) %>% 
  select(country, region, sub_region)

region <- bind_rows(region, corrected)

sample_n(region, 20)

country,region,sub_region
<chr>,<chr>,<chr>
"Palestine, State of",Asia,Western Asia
New Zealand,Oceania,Australia and New Zealand
"Serbia, Republic of",Europe,Southern Europe
Egypt,Africa,Northern Africa
United States,Americas,Northern America
Costa Rica,Americas,Latin America and the Caribbean
Eswatini,Africa,Sub-Saharan Africa
Mozambique,Africa,Sub-Saharan Africa
Botswana,Africa,Sub-Saharan Africa


Now the region dataframe is cleaned. I join the financial dataframe with the region dataframe. Then I control that there are no missing countries. As can be seen below there are none except for those regions we purposely left out.

In [21]:
financial_data2 <- left_join(financial_data, region, by=c("country"))

catn("Countries with no match in region dataframe:")
financial_data2 %>% select(country, region) %>% 
  filter_all(any_vars(is.na(.))) %>% 
  distinct()



Countries with no match in region dataframe:



country,region
<chr>,<chr>
Advanced Economies,NA
CIS,NA
Emerging and Developing Asia,NA
Emerging and Developing Countries,NA
Emerging and Developing Europe,NA
Europe,NA
"Middle East, North Africa, Afghanistan, and Pakistan",NA
Sub-Saharan Africa,NA
Western Hemisphere,NA


Now I need to spread the financial dataframe to get all the kpis in separate columns. I only use the time period month. Personally, I like to have a column that contains both year and period. Therefore I create a new column called year_period. This column is based on the year and month column. To do thies I use the [pull](https://dplyr.tidyverse.org/reference/pull.html) function from dplyr in combination with the [unite](https://tidyr.tidyverse.org/reference/unite.html) function from tidyr.

In [22]:
imf <- financial_data2 %>% spread(key = kpi, value = value) %>% 
  filter(time_period == "month") %>% 
  select(-time_period)
year_period <- pull(imf %>% unite(year_period, year, period, sep = "-") %>% 
  select(year_period))


imf <- imf %>% add_column(year_period, .after=3)
sample_n(imf, 10)

country,year,period,year_period,region,sub_region,economic_activity_industrial_production_index,employment_persons_number_of,financial_market_prices_equities_end_of_period_index,labor_force_persons_number_of,labor_markets_unemployment_rate_percent,prices_consumer_price_index_all_items_index,prices_producer_price_index_all_commodities_index,unemployment_persons_number_of
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Paraguay,2018,09,2018-09,Americas,Latin America and the Caribbean,NA,NA,NA,NA,NA,140.4316,NA,NA
"Timor-Leste, Dem. Rep. of",2016,03,2016-03,Asia,South-eastern Asia,NA,NA,NA,NA,NA,141.1412,NA,NA
Curaçao,2015,03,2015-03,Americas,Latin America and the Caribbean,NA,NA,NA,NA,NA,107.0632,NA,NA
Suriname,2013,07,2013-07,Americas,Latin America and the Caribbean,NA,NA,NA,NA,NA,126.6754,NA,NA
Cameroon,2015,02,2015-02,Africa,Sub-Saharan Africa,NA,NA,NA,NA,NA,111.8387,NA,NA
"Armenia, Republic of",2015,04,2015-04,Asia,Western Asia,NA,NA,NA,NA,NA,127.1319,134.4266,74.2
Slovak Republic,2018,12,2018-12,Europe,Eastern Europe,NA,NA,NA,NA,NA,112.6227,NA,NA
Burundi,2013,11,2013-11,Africa,Sub-Saharan Africa,NA,NA,NA,NA,NA,141.7430,NA,NA
Lao People's Democratic Republic,2017,12,2017-12,Asia,South-eastern Asia,NA,NA,NA,NA,NA,129.4961,NA,NA


Now the dataframe is tidy. Now I need to save it.

## 4. Save data

In [24]:
write.csv(imf, "imf_monthly_r.csv")